<a href="https://colab.research.google.com/github/Deepshika-286/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import json
import numpy as np


import tensorflow as tf
from tensorflow import keras
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D


import pickle
import random

from sklearn.metrics import precision_score, f1_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# data

In [ ]:
f = open('/content/data.json')
data = json.load(f)
data

{'intents': [{'tag': 'Cuts',
   'patterns': ['What to do if Cuts?',
    'How to cure Cuts?',
    'Which medicine to apply for Cuts?',
    'what to apply on cuts?',
    'Cuts'],
   'responses': ['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.'],
   'context_set': ''},
  {'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Whats up'],
   'responses': ['Hi , i am your paramedic assistant'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day',
    'bye'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'context_set': ''},
  {'tag': 'Abrasions',
   'patterns': ['how do you treat abrasions

In [ ]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [ ]:
training_sentences[:5]

['What to do if Cuts?',
 'How to cure Cuts?',
 'Which medicine to apply for Cuts?',
 'what to apply on cuts?',
 'Cuts']

In [ ]:
training_labels

array([ 9,  9,  9,  9,  9, 41, 41, 41, 41, 41, 40, 40, 40, 40, 40, 40,  1,
        1,  1,  1,  1,  1, 45, 45, 45, 45, 31, 31, 31, 31, 32, 32, 32, 32,
       33, 33, 33, 33, 33, 33, 14, 14, 14, 14, 23, 23, 23, 23, 23, 23,  8,
        8,  8,  8,  8, 30, 30, 30, 30, 17, 17, 17, 17, 29, 29, 29, 29,  0,
        0,  0,  0,  3,  3,  3,  3,  2,  2,  2,  2,  6,  6,  6,  6, 38, 38,
       38, 38, 10, 10, 10, 10, 16, 16, 16, 16, 20, 20, 20, 20, 21, 21, 21,
       21, 22, 22, 22, 22, 42, 42, 42, 42, 26, 26, 26, 26, 28, 28, 28, 28,
       34, 34, 34, 34, 36, 36, 36, 36, 37, 37, 37, 37, 24, 24, 24, 24, 12,
       12, 12, 12,  5,  5,  5,  5, 25, 25, 25, 25, 35, 35, 35, 35, 43, 43,
       43, 43, 18, 18, 18, 18, 13, 13, 13, 13, 19, 19, 19, 19,  7,  7,  7,
        7, 27, 27, 27, 27, 44, 44, 44, 44, 44, 39, 39, 39, 39, 39, 39, 39,
       11, 11, 11, 11,  4,  4,  4,  4, 15, 15, 15, 15])

In [ ]:
print(len(training_labels))

199


In [ ]:
labels[:10]

['Cuts',
 'greeting',
 'goodbye',
 'Abrasions',
 'stings',
 'Splinter',
 'Sprains',
 'Strains',
 'Fever',
 'Nasal Congestion']

In [ ]:
print(len(labels))

46


In [ ]:
responses[:10]

[['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.'],
 ['Hi , i am your paramedic assistant'],
 ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
 ['Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth or bandage, and apply gentle pressure to the area to stop any bleeding.Cover a wound that bled with a thin layer of topical antibiotic ointment, like Bacitracin, or a sterile moisture barrier ointment, like Aquaphor. Cover it with a clean bandage or gauze. Gently clean the wound and change the ointment and bandage once

# data preprocessing

In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)
training_labels

array([ 9,  9,  9,  9,  9, 41, 41, 41, 41, 41, 40, 40, 40, 40, 40, 40,  1,
        1,  1,  1,  1,  1, 45, 45, 45, 45, 31, 31, 31, 31, 32, 32, 32, 32,
       33, 33, 33, 33, 33, 33, 14, 14, 14, 14, 23, 23, 23, 23, 23, 23,  8,
        8,  8,  8,  8, 30, 30, 30, 30, 17, 17, 17, 17, 29, 29, 29, 29,  0,
        0,  0,  0,  3,  3,  3,  3,  2,  2,  2,  2,  6,  6,  6,  6, 38, 38,
       38, 38, 10, 10, 10, 10, 16, 16, 16, 16, 20, 20, 20, 20, 21, 21, 21,
       21, 22, 22, 22, 22, 42, 42, 42, 42, 26, 26, 26, 26, 28, 28, 28, 28,
       34, 34, 34, 34, 36, 36, 36, 36, 37, 37, 37, 37, 24, 24, 24, 24, 12,
       12, 12, 12,  5,  5,  5,  5, 25, 25, 25, 25, 35, 35, 35, 35, 43, 43,
       43, 43, 18, 18, 18, 18, 13, 13, 13, 13, 19, 19, 19, 19,  7,  7,  7,
        7, 27, 27, 27, 27, 44, 44, 44, 44, 44, 39, 39, 39, 39, 39, 39, 39,
       11, 11, 11, 11,  4,  4,  4,  4, 15, 15, 15, 15])

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
print(padded_sequences)

[[ 0  0  0 ...  3  6 27]
 [ 0  0  0 ...  2 15 27]
 [ 0  0  0 ... 19 65 27]
 ...
 [ 0  0  0 ...  8  4 64]
 [ 0  0  0 ... 18  4 64]
 [ 0  0  0 ... 15  4 64]]


# model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# training

In [ ]:
epochs = 250
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 210ms/step - accuracy: 0.0234 - loss: 3.8290
Epoch 2/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0106 - loss: 3.8278      
Epoch 3/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0266 - loss: 3.8270 
Epoch 4/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0081 - loss: 3.8272      
Epoch 5/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0391 - loss: 3.8250 
Epoch 6/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0271 - loss: 3.8253 
Epoch 7/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0341 - loss: 3.8242 
Epoch 8/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0229 - loss: 3.8234     
Epoch 9/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0297 - loss: 3.8213     
Epoch 10/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0385 - loss: 3.8175 
Epoch 11/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0179 - loss: 3.8219     
Epoch 12/250
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step -

# chatbot

In [ ]:
model.save("chat_model.h5")

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("data.json") as file:
    data = json.load(file)
def chat():
    # load trained model
    model = keras.models.load_model('chat_model.h5') # Add the .h5 extension here

    # ... (rest of the code remains the same)

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        print( "User: ", end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print ("ChatBot:" , np.random.choice(i['responses']))
print( "Hii, Iam chatbot, I can help u with")
for i in labels:
  print(i, end = ",   ")
print("\n \n Start messaging with the bot (type quit to stop)! \n ")

chat()

Hii, Iam chatbot, I can help u with
Cuts,   greeting,   goodbye,   Abrasions,   stings,   Splinter,   Sprains,   Strains,   Fever,   Nasal Congestion,   Cough,   Sore Throat,   Gastrointestinal problems,   Skin problems,   Abdonominal Pain,   Bruises,   Broken Toe,   Choking,   Wound,   Diarrhea,   Frost bite,   Heat Exhaustion,   Heat Stroke,   Insect Bites,   nose bleed,   Pulled Muscle,   Rectal bleeding,   Sun Burn,   Testicle Pain,   Vertigo,   Normal Bleeding,   Eye Injury,   Chemical Burn,   Poison,   Teeth,   seizure,   Head Injury,   Fainting,   Headache,   Cold,   Rash,   snake bite,   animal bite,   Drowning,   CPR,   Fracture,   
 
 Start messaging with the bot (type quit to stop)! 
 


User: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
ChatBot: Hi , i am your paramedic assistant
User: cuts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
ChatBot: Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.
User: sprain
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
ChatBot: Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth or bandage, and apply gentle pressure to the area to stop any bleeding.Cover a wound that bled with a thin layer of topical antibiotic ointment, like Bacitracin, or a sterile moisture barrier ointment, like Aquaphor. Cov